- fastapi+react생성 : https://blog.naver.com/blueberrytomato/223267540212

# m1 mac react 설치
https://velog.io/@gnwjd309/%EB%A7%A5%EB%B6%81-m1-%EB%A6%AC%EC%95%A1%ED%8A%B8-%EA%B0%9C%EB%B0%9C%ED%99%98%EA%B2%BD-%EB%B9%A0%EB%A5%B4%EA%B2%8C-%EA%B5%AC%EC%B6%95%ED%95%98%EA%B8%B0

# FastAPI 폴더 생성

# 파이썬 가상환경 생성

In [ ]:
python -m venv venv

# 가상환경 활성화

In [ ]:
source venv/bin/activate

# fastapi 설치

In [ ]:
pip install fastapi uvicorn sqlalchemy

# FastAPI 디렉토리에 database.py, models.py, main.py파일 생성

# database.py 파일 설정

In [ ]:
####데이터베이스 설정과 연결 구성####
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
# 데이터베이스 URL 설정
URL_DATABASE = "sqlite:///./finance.db"
# sqlalchemy 엔진 생성(데이터베이스 연결 관리)
engine=create_engine(URL_DATABASE, connect_args={'check_same_thread':False})
# 세션 팩토리 설정(데이터베이스 작성 세션 생성)
SessionLocal=sessionmaker(autocommit=False, autoflush=False, bind=engine)
# 모델의 베이스 클래스(데이터베이스 테이블, 모델 클래스 연결)
Base=declarative_base()

# models.py 파일 설정

In [ ]:
from database import Base
from sqlalchemy import Column, Integer, String, Boolean, Float
class Transaction(Base):
   __tablename__="chatgpt" #테이블 이름 설정
   id=Column(Integer, primary=True, index=True) #기본 키
   chatbot=Column(String) #챗봇
   title=Column(String) #제목
   description=Column(String) #질문
   date=Column(String) #날짜

# main.py 파일 설정

In [ ]:
# 웹사이트를 구축하는 주요 파일 #라우트, 모델 정의 ,데이터베이스 세션 관리
from fastapi import FastAPI, HTTPException, Depends
from typing import Annotated, List
from sqlalchemy.orm import Session
from pydantic import BaseModel
from database import SessionLocal, engine
import models
from fastapi.moddleware.cors import CORSMiddleware


app=FastAPI()


#CORS(프론트=>백엔드 접근 설정) 설정
origins=["https://localhost:3000"]
app.add_middleware(
   CORSMiddleware,
   allow_origins=origins,
   allow_credentials=True,
   allow_methods=['*'],
   allow_headers=['*'],
)


#Pydantic 모델 정의
class chatgptBase(BaseModel):
   chatbot:str
   title:str
   description:str
   date:str

class chatgptModel(chatgptBase):
   id:int
   class Config:
       orm_mode=True #ORM 모드 설정


def get_db():
   db=SessionLocal()
   try:
       yield db
   finally:
       db.close() #작업 종료 후 세션 닫기

db_dependency=Annotated[Session, Depends(get_db)]
models.Base.metadata.create_all(bind=engine) #데이터베이스 테이블 생성


#API 엔드포인트
@app.post("/chatgpt/",response_model=chatgptModel)
async def create_chatgpt(transaction:chatgptBase, db:db_dependency):
   db_transaction=models.Transaction(**transaction.dict())
   db.add(db_transaction)
   db.commit()
   db.refresh(db_transaction)
   return db_transaction


@app.get("/chatgpt/",response_model=List[chatgptModel])
async def read_chatgpt(db:db_dependency, skip: int=0, limit: int=100):
   transactions=db.query(models.Transaction).offset(skip).limit(limit).all()
   return transactions

# FastAPI 정상 작동 테스트

In [ ]:
cd FastAPI

In [ ]:
uvicorn main:app —-reload
# http://127.0.0.1:8000/docs 접속

# React 구성

In [ ]:
cd ./React/

In [ ]:
npx-create-react-app my_app

# scr폴더에 api.js 생성 후 설정

In [ ]:
// React 프론트엔드 애플리케이션에서 백엔드 API에 접근하기 위해 걸정된 파일
import axios from 'axios'

// HTTP 요청 처리를 위한 기본 설정포함
const api=axios.craete({
    baseURL:'https://localhost:8000', //FastAPI 백엔드 서버 주소
});

export default api; //생성된 인스턴트 내보내기

# App.js 파일 설정

In [ ]:
import React, {useState, useEffect} from 'react';
import api from './Api'
// import logo from './logo.svg';
import './App.css';

const App = () => {
  const [chatgpts, setchatgpts]=useState([]);
  const [formData, setFormData]=useState({
    chatbot:'',
    title:'',
    description:'',
    date:''
  });
  const fetchchatgpt=async()=>{
    const response=await api.get('/chatgpt/');
    setchatgpts(response.data)
  }
  useEffect(()=>{
    fetchchatgpt();
  },[]);
  const handleInputchange=(event)=>{
    const value=event.target.type==='checkout' ? event.target.checked : event.target.value;
    setFormData({
      ...formData,
      [event.target.name]: value
  })
  };
  const handleFormSubmit=async(event)=>{
    event.preventDefault();
    await api.post('/chatgpt/',formData);
    fetchchatgpt();
    setFormData({
      chatbot:'',
      title:'',
      description:'',
      date:''
    })
  }
  return (
    <div>
      <nav className='navbar navbar-dark bg-primary'>
        <div className='container-fluid'>
          <a className='navbar-brand' href="#">
            Hello, this is recurise world!
          </a>
        </div>
      </nav>

      <div className="container">
        <form onSubmit={handleFormSubmit}>
          <div className='mb-3 mt-3'>
            <label htmlFor='chatbot' className='form-label'>
              chatbot
            </label>
            <input type='text' className='form-control' id ='chatbot' name='chatbot' onChange={handleInputchange} value={formData.chatbot}></input>
          </div>
          <div className='mb-3'>
            <label htmlFor='title' className='form-label'>
              title
            </label>
            <input type='text' className='form-control' id ='title' name='title' onChange={handleInputchange} value={formData.title}></input>
          </div>
          <div className='mb-3'>
            <label htmlFor='description' className='form-label'>
              description
            </label>
            <input type='text' className='form-control' id ='description' name='description' onChange={handleInputchange} value={formData.description}></input>
          </div>
          <div className='mb-3'>
            <label htmlFor='date' className='form-label'>
              date
            </label>
            <input type='text' className='form-control' id ='date' name='date' onChange={handleInputchange} value={formData.date}></input>
          </div>
          <button type='submit' className='btn btn-primary'>
            Submit
          </button>
        </form>

        <table className='table table-striped table-bordered table-hover'>
          <thread>
            <tr>
              <th>chatbot</th>
              <th>title</th>
              <th>description</th>
              <th>date</th>
            </tr>
          </thread>
          <tbody>
            {chatgpts.map((chatgpt)=>(
              <tr key={chatgpt.id}>
                <td>{chatgpt.chatbot}</td>
                <td>{chatgpt.title}</td>
                <td>{chatgpt.description}</td>
                <td>{chatgpt.date}</td>
              </tr>
            ))}
          </tbody>
        </table>
      </div>
    </div>
  );
}

export default App;


# index.html에 링크 복붙

In [ ]:
<link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@4.4.1/dist/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<script src="https://cdn.jsdelivr.net/npm/popper.js@1.16.0/dist/umd/popper.min.js" integrity="sha384-Q6E9RHvbIyZFJoft+2mJbHaEWldlvI9IOYy5n3zV9zzTtmI3UksdQRVvoxMfooAo" crossorigin="anonymous"></script>

# axios 설치 밑 React 실행

In [ ]:
npm install axios

In [ ]:
npm start

# 페이지 주소 : http://localhost:3000/#

-오라클 클라우드 vm 우분투 서버에 올리기 : https://m.blog.naver.com/blueberrytomato/223270502515

In [ ]:
server {
  listen 80 default_server;
  listen [::]:80 default_server;
  server_name chatgpt;
location / {
  proxy_pass http://localhost:3000;
  proxy_http_version 1.1;
  proxy_set_header Upgrade $http_upgrade;
  proxy_set_header Connection 'upgrade';
  proxy_set_header Host $host;
  proxy_cache_bypass $http_upgrade;
}

location /api{
  rewrite ^/api/(.*)$ /$1 break;
  proxy_pass http://localhost:8000;
  proxy_http_version 1.1;
  proxy_set_header Upgrade $http_upgrade;
  proxy_set_header Connection 'upgrade';
  proxy_set_header Host $host;
  proxy_cache_bypass $http_upgrade;
}}